In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = 4

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows={
            'train': {
                1: ("2014-01-01", "2014-02-01")
            },
            'unseen_activations_of_seen_appliances': {
                1: ("2014-02-02", "2014-02-08")                
            },
            'unseen_appliances': {
                2: ("2013-06-01", "2013-06-07")
            }
        }
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 112 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 119 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 30 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [3]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [4]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False
)

In [14]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [15]:
all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

True

In [16]:
fig, axes = plt.subplots(2, sharex=True)
seq.all_appliances.plot(ax=axes[0])
axes[1].plot(seq.input)
fig.suptitle(FOLD)
plt.show()

In [17]:
sample = source.get_batch(num_seq_per_batch=1024)

In [18]:
pipeline = DataPipeline(
    [source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std())],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

In [ ]:
nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

In [ ]:
disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

In [ ]:
disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [19]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
   'allow_incomplete_target': False,
   'distractor_inclusion_prob': 1.0,
   'include_incomplete_target_in_output': True,
   'rng_seed': None,
   'seq_length': 256,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 1.0,
   'uniform_prob_of_selecting_each_model': True},
  'input_processing': [{'DivideBy': {'divisor': 1001.1713}}],
  'num_seq_per_batch': 64,
  'target_processing': [{'DivideBy': {'divisor': 607.55792}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}}}}

In [20]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [21]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [22]:
from pymongo import MongoClient
client = MongoClient()
client.drop_database('neuralnilm_experiments')
db = client.neuralnilm_experiments
experiment_collection = db.e1

In [23]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net,
    data_pipeline=pipeline,
    collection=experiment_collection,
    metrics=Metrics([4]),
    learning_rate=1E-2,
    repeat_callbacks=[
        ( 100, Trainer.validate)
    ]
)

INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02


In [24]:
trainer.fit(1000)

INFO:neuralnilm.trainer:Starting training for 1000 iterations.
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.
INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Stopped training. Completed 1000 iterations.


In [25]:
trainer.iteration

1000

In [26]:
import pymongo
list(experiment_collection.scores.train.find().sort('_id', pymongo.DESCENDING))

[{u'_id': 999, u'loss': 0.42587241530418396, u'source_id': 0},
 {u'_id': 998, u'loss': 0.46156948804855347, u'source_id': 0},
 {u'_id': 997, u'loss': 0.4434300661087036, u'source_id': 0},
 {u'_id': 996, u'loss': 0.3990124464035034, u'source_id': 0},
 {u'_id': 995, u'loss': 0.3984493613243103, u'source_id': 0},
 {u'_id': 994, u'loss': 0.39107412099838257, u'source_id': 0},
 {u'_id': 993, u'loss': 0.4012400209903717, u'source_id': 0},
 {u'_id': 992, u'loss': 0.43601635098457336, u'source_id': 0},
 {u'_id': 991, u'loss': 0.41309139132499695, u'source_id': 0},
 {u'_id': 990, u'loss': 0.400470495223999, u'source_id': 0},
 {u'_id': 989, u'loss': 0.43309423327445984, u'source_id': 0},
 {u'_id': 988, u'loss': 0.4561264216899872, u'source_id': 0},
 {u'_id': 987, u'loss': 0.44510260224342346, u'source_id': 0},
 {u'_id': 986, u'loss': 0.4867698848247528, u'source_id': 0},
 {u'_id': 985, u'loss': 0.42117685079574585, u'source_id': 0},
 {u'_id': 984, u'loss': 0.44372522830963135, u'source_id': 0},


In [27]:
list(experiment_collection.scores.validation.find())

[{u'SyntheticAggregateSource': {u'train': {u'2_state_classification': {u'accuracy_score': 0.998779296875,
     u'f1_score': 0.0,
     u'precision_score': 0.0,
     u'recall_score': 0.0},
    u'regression': {u'mean_absolute_error': 0.005146272946149111,
     u'relative_error_in_total_energy': -0.9472641944885254}},
   u'unseen_activations_of_seen_appliances': {u'2_state_classification': {u'accuracy_score': 0.99969482421875,
     u'f1_score': 0.0,
     u'precision_score': 0.0,
     u'recall_score': 0.0},
    u'regression': {u'mean_absolute_error': 0.0012827713508158922,
     u'relative_error_in_total_energy': -1.0}},
   u'unseen_appliances': {u'2_state_classification': {u'accuracy_score': 0.885498046875,
     u'f1_score': 0.0,
     u'precision_score': 0.0,
     u'recall_score': 0.0},
    u'regression': {u'mean_absolute_error': 0.5573543310165405,
     u'relative_error_in_total_energy': -1.0}}},
  u'_id': 0},
 {u'SyntheticAggregateSource': {u'train': {u'2_state_classification': {u'accurac